<a href="https://colab.research.google.com/github/TrungBui-Purdue-5913/Spotify-Analysis_SIGAI2021/blob/main/Musics_recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# import libraries

import pandas as pd
import numpy as np

In [4]:
# load data
df = pd.read_csv('./data/data.csv')

# (1) DATA EXPLORATION & CLEANING

In [5]:
df.head()

,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo,valence,year
0,0.991000,['Mamie Smith'],0.598,168333,0.224,0,0cS0A1fUEUd1EW3FcF8AEI,0.000522,5,0.3790,-12.628,0,Keep A Song In Your Soul,12,1920,0.0936,149.976,0.6340,1920
1,0.643000,"[""Screamin' Jay Hawkins""]",0.852,150200,0.517,0,0hbkKFIJm7Z05H8Zl9w30f,0.026400,5,0.0809,-7.261,0,I Put A Spell On You,7,1920-01-05,0.0534,86.889,0.9500,1920
2,0.993000,['Mamie Smith'],0.647,163827,0.186,0,11m7laMUgmOKqI3oYzuhne,0.000018,0,0.5190,-12.098,1,Golfing Papa,4,1920,0.1740,97.600,0.6890,1920
3,0.000173,['Oscar Velazquez'],0.730,422087,0.798,0,19Lc5SfJJ5O1oaxY0fpwfh,0.801000,2,0.1280,-7.311,1,True House Music - Xavier Santos & Carlos Gomi...,17,1920-01-01,0.0425,127.997,0.0422,1920
4,0.295000,['Mixe'],0.704,165224,0.707,1,2hJjbsLCytGsnAHfdsLejp,0.000246,10,0.4020,-6.036,0,Xuniverxe,2,1920-10-01,0.0768,122.076,0.2990,1920


In [6]:
df.info()

# no null values are found

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 174389 entries, 0 to 174388
Data columns (total 19 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   acousticness      174389 non-null  float64
 1   artists           174389 non-null  object 
 2   danceability      174389 non-null  float64
 3   duration_ms       174389 non-null  int64  
 4   energy            174389 non-null  float64
 5   explicit          174389 non-null  int64  
 6   id                174389 non-null  object 
 7   instrumentalness  174389 non-null  float64
 8   key               174389 non-null  int64  
 9   liveness          174389 non-null  float64
 10  loudness          174389 non-null  float64
 11  mode              174389 non-null  int64  
 12  name              174389 non-null  object 
 13  popularity        174389 non-null  int64  
 14  release_date      174389 non-null  object 
 15  speechiness       174389 non-null  float64
 16  tempo             17

In [7]:
df.describe()

# There's some values that are 0 such as tempo, which should be taken with a grain of salt.

,acousticness,danceability,duration_ms,energy,explicit,instrumentalness,key,liveness,loudness,mode,popularity,speechiness,tempo,valence,year
count,174389.000000,174389.000000,1.743890e+05,174389.000000,174389.000000,174389.000000,174389.000000,174389.000000,174389.000000,174389.000000,174389.000000,174389.000000,174389.000000,174389.000000,174389.000000
mean,0.499228,0.536758,2.328100e+05,0.482721,0.068135,0.197252,5.205305,0.211123,-11.750865,0.702384,25.693381,0.105729,117.006500,0.524533,1977.061764
std,0.379936,0.176025,1.483958e+05,0.272685,0.251978,0.334574,3.518292,0.180493,5.691591,0.457211,21.872740,0.182260,30.254178,0.264477,26.907950
min,0.000000,0.000000,4.937000e+03,0.000000,0.000000,0.000000,0.000000,0.000000,-60.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1920.000000
25%,0.087700,0.414000,1.661330e+05,0.249000,0.000000,0.000000,2.000000,0.099200,-14.908000,0.000000,1.000000,0.035200,93.931000,0.311000,1955.000000
50%,0.517000,0.548000,2.057870e+05,0.465000,0.000000,0.000524,5.000000,0.138000,-10.836000,1.000000,25.000000,0.045500,115.816000,0.536000,1977.000000
75%,0.895000,0.669000,2.657200e+05,0.711000,0.000000,0.252000,8.000000,0.270000,-7.499000,1.000000,42.000000,0.076300,135.011000,0.743000,1999.000000
max,0.996000,0.988000,5.338302e+06,1.000000,1.000000,1.000000,11.000000,1.000000,3.855000,1.000000,100.000000,0.971000,243.507000,1.000000,2021.000000


In [8]:
for i in df.columns:
    if df[i].dtype == "float64" or df[i].dtype == "int64":
        if df.describe().loc['min',i] == 0:
            print(i + ": " + df[df[i] == 0].head(1)['name'])
            
# Dataset is not free of errors
# (1) The danceability score of 0 of "La Mina del Ford - Remasterizado" is questionable
# (2) Song like "Oouin" has instruments but says instrumentalness = 0
# (3) Songs like "La Mina del Ford - Remasterizado" have a tempo

18946    acousticness: Brown Noise - Loopable with No Fade
Name: name, dtype: object
758    danceability: La Mina del Ford - Remasterizado
Name: name, dtype: object
23004    energy: Pause Track - Live
Name: name, dtype: object
0    explicit: Keep A Song In Your Soul
Name: name, dtype: object
10    instrumentalness: Oouin
Name: name, dtype: object
2    key: Golfing Papa
Name: name, dtype: object
18946    liveness: Brown Noise - Loopable with No Fade
Name: name, dtype: object
0    mode: Keep A Song In Your Soul
Name: name, dtype: object
7    popularity: Arkansas Blues
Name: name, dtype: object
758    speechiness: La Mina del Ford - Remasterizado
Name: name, dtype: object
758    tempo: La Mina del Ford - Remasterizado
Name: name, dtype: object
758    valence: La Mina del Ford - Remasterizado
Name: name, dtype: object


In [9]:
df.corr()

# no specifically high correlation (besides loudness and energy 0.779)

,acousticness,danceability,duration_ms,energy,explicit,instrumentalness,key,liveness,loudness,mode,popularity,speechiness,tempo,valence,year
acousticness,1.000000,-0.263217,-0.089169,-0.750852,-0.208176,0.221956,-0.028028,-0.029654,-0.546639,0.064633,-0.396744,-0.022437,-0.223840,-0.166968,-0.607515
danceability,-0.263217,1.000000,-0.100757,0.204838,0.200842,-0.215589,0.026266,-0.110033,0.249541,-0.048358,0.123746,0.239962,0.005479,0.536713,0.159095
duration_ms,-0.089169,-0.100757,1.000000,0.060516,-0.033808,0.103621,0.002020,0.028942,0.019791,-0.046849,0.024717,-0.097838,-0.008182,-0.183199,0.105661
energy,-0.750852,0.204838,0.060516,1.000000,0.102561,-0.177750,0.035780,0.134815,0.779267,-0.056160,0.328939,-0.112616,0.266448,0.326418,0.540850
explicit,-0.208176,0.200842,-0.033808,0.102561,1.000000,-0.130609,0.005282,0.037288,0.106249,-0.062503,0.152545,0.353872,0.008075,-0.009275,0.151619
instrumentalness,0.221956,-0.215589,0.103621,-0.177750,-0.130609,1.000000,-0.004619,-0.047941,-0.317562,-0.056731,-0.300625,-0.133966,-0.068656,-0.219188,-0.114259
key,-0.028028,0.026266,0.002020,0.035780,0.005282,-0.004619,1.000000,-0.003368,0.025227,-0.127397,0.001951,0.009648,0.005009,0.025592,0.019194
liveness,-0.029654,-0.110033,0.028942,0.134815,0.037288,-0.047941,-0.003368,1.000000,0.062695,0.001677,-0.078959,0.122034,0.008586,-0.005781,-0.011852
loudness,-0.546639,0.249541,0.019791,0.779267,0.106249,-0.317562,0.025227,0.062695,1.000000,-0.019250,0.337194,-0.213504,0.217914,0.302520,0.465189
mode,0.064633,-0.048358,-0.046849,-0.056160,-0.062503,-0.056731,-0.127397,0.001677,-0.019250,1.000000,0.007652,-0.040711,0.002438,0.021592,-0.048922


# (1.2) Drop irrelevant features

In [82]:
df_sample = df.sample(n=10000)

# dropping song id
song_id = df_sample['id']
df_new = df_sample.drop('id', axis=1)

# Temporarily drop title
df_new = df_new.drop('name', axis=1)
# drop release date
df_new = df_new.drop('release_date', axis=1)

# converting column values of "artists"
df_new['artists'] = df_new['artists'].str.replace("'", " ")
df_new['artists'] = df_new['artists'].str.replace("[", " ")
df_new['artists'] = df_new['artists'].str.replace("]", " ")

df_new = pd.concat([df_new.drop('artists', axis=1), df_new['artists'].str.get_dummies(sep=',')], axis=1)

# (2) Training ML Algorithm, Diagnosis

In [83]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=50).fit_predict(df_new)

In [97]:
df_sample['kmeans'] = kmeans

In [128]:
def give_rec(song_id_inputs):
    
    df_sample['id'] = song_id
    
    k_dict = {}
    for i in df_sample.loc[df_sample['id'].isin(song_id_inputs)]['kmeans']:
        if i not in k_dict.keys():
            k_dict[i] = 1
        else:
            k_dict[i] += 1
    k = max(k_dict, key=k_dict.get)

    recommendations = df_sample[df_sample["kmeans"] == k]
    sampled = recommendations.sample(10)
    return df_sample.loc[df_sample['id'].isin(sampled['id'])][['id', 'artists', 'name']]

In [129]:
df_sample.sort_values("popularity", ascending=False)[:10]

,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo,valence,year,kmeans
19870,0.1900,['CJ'],0.711,123429,0.598,1,5vGLcdRuSbUhD8ScwsGSdA,0.000000,3,0.1500,-7.621,0,Whoopty,94,2020-08-20,0.1820,140.175,0.737,2020,37
19924,0.0676,"['Shawn Mendes', 'Justin Bieber']",0.652,178994,0.383,0,2Z8yfpFX0ZMavHkcIeHiO1,0.000000,2,0.0828,-7.076,0,Monster (Shawn Mendes & Justin Bieber),91,2020-11-20,0.0516,145.765,0.549,2020,2
19864,0.4990,['SZA'],0.436,279204,0.655,1,3YJJjQPAbDT7mGpX3WtQ9A,0.000008,1,0.6880,-8.370,0,Good Days,91,2020-12-25,0.0583,121.002,0.412,2020,90
19996,0.2340,['Bad Bunny'],0.811,130014,0.637,1,5RubKOuDoPn5Kj5TLVxSxY,0.000572,10,0.1180,-4.835,0,TE MUDASTE,90,2020-11-27,0.0591,92.025,0.471,2020,60
93700,0.1670,['Camilo'],0.824,187427,0.457,0,73nAK3HgQK8dak83Y2WQ8F,0.000000,6,0.0410,-5.428,1,Vida de Rico,90,2020-09-21,0.0543,87.977,0.950,2020,52
126583,0.0165,['Justin Quiles'],0.807,185840,0.656,1,3GtnId3Cm1Xa6joCj08nzW,0.000000,3,0.0693,-4.395,0,Jeans,88,2020-08-14,0.1050,83.943,0.890,2020,40
19944,0.3600,['Ariana Grande'],0.487,201882,0.534,0,3UoULw70kMsiVXxW0L3A33,0.000000,0,0.1000,-5.664,0,pov,87,2020-10-30,0.0555,131.798,0.173,2020,94
19824,0.2950,"['KAROL G', 'Nicki Minaj']",0.803,200960,0.715,0,7k4t7uLgtOxPwTpFmtJNTY,0.000134,2,0.0574,-3.280,1,Tusa,87,2019-11-07,0.2980,101.085,0.574,2019,94
19720,0.2560,['Doja Cat'],0.787,237893,0.673,1,3Dv1eDb0MEgF93GpLXlucZ,0.000004,11,0.0904,-4.577,0,Say So,86,2019-11-07,0.1580,110.962,0.786,2019,75
19704,0.9020,['Billie Eilish'],0.704,245426,0.225,0,3ZCTVFBt2Brf31RLEnCkWJ,0.657000,6,0.1060,-14.454,0,everything i wanted,86,2019-11-13,0.0994,120.006,0.243,2019,43


In [134]:
id_list = list(df_sample.sort_values("popularity", ascending=False)[:10]["id"])
print(give_rec(id_list))

                            id  \
164072  0YMxpwBUCzAFjx0KxohNc8   
29289   6QiRFf6spEpBxAw2J26tAM   
43909   1A0uDAyazt1JiyiBtUZBoD   
117812  0kFPDmoDMu4dpVCvTeuMSl   
173043  1BTj47Up5m8601KOFrTvkj   
12816   0uLI1jac8ZJSSRG4QJDo3J   
63146   08vwwsifRi8U4wjbjJKBml   
3011    7c2WEZ4wFacBfPA55VGbet   
40912   3HA0j1OxxFZS2Iix5ZQ6oz   
94573   4Hxm08tWVyqpFzHlFl55XV   

                                                  artists  \
164072                          ['Gunter Kallmann Choir']   
29289                            ['Donovan', 'Jeff Beck']   
43909                                         ['Girdhar']   
117812  ['Alfredo Gutiérrez y sus Acordeones Dorados',...   
173043                                          ['Logic']   
12816                                      ['The Smiths']   
63146              ['Chet Baker Quartet', 'Russ Freeman']   
3011                                      ['Umm Kulthum']   
40912                                      ['Guandulito']   
94573          

# (3) Hyperparameter Tuning and Validation